In [ ]:
# Import necessary modules.
import time
import numpy as np
import pandas as pd
from od_cal_cls.fdsa import fdsa
import plotly.graph_objects as go

In [ ]:
# Set up plotting templete.
tempelete_01_white = dict(
    layout = go.Layout(
        # Layout properties
        title_font_size= 14,
        title_x= 0.1,
        font_size= 11,
        font_color= "#000000",
        font_family= "Times New Roman",
        margin_b = 65,
        margin_l = 60,
        margin_r = 30,
        margin_t = 50,
        plot_bgcolor= "#ffffff",
        # X axis properties
        xaxis_color= "#000000",
        xaxis_linecolor= "#000000",
        xaxis_ticks= "inside",        
        xaxis_tickfont_color= "#000000",
        xaxis_tickfont_family= "Times New Roman",
        xaxis_mirror= True,
        xaxis_showline= True,
        xaxis_showgrid= False,
        # Y axis properties
        yaxis_color= "#000000",
        yaxis_linecolor= "#000000",
        yaxis_ticks= "inside",
        yaxis_tickfont_color= "#000000",
        yaxis_tickfont_family= "Times New Roman",
        yaxis_mirror= True,
        yaxis_showline= True,
        yaxis_showgrid= False,
    )
)

In [ ]:
# Visualize & Check step value.
def eval_step(x, step_ini, param_A, param_alpha):
    step = step_ini / ((x + param_A)**param_alpha)
    return step

iter_opti = list(range(1,201))
step = list(map(lambda x: eval_step(x, step_ini= 100, param_A= 9, param_alpha= 0.2), iter_opti))

fig_step = go.Figure()

fig_step.add_trace(
    go.Scatter(
        x= iter_opti,
        y= step,
        line_color = "#000000",
    )
)

fig_step.update_layout(
    title= "Step Changes",
    xaxis_title= "Number Iteration",
    yaxis_title= "Step Value [NrVeh/hr]",
    width= 500,
    height= 350,
    template= tempelete_01_white,
)

fig_step.update_xaxes(
    range= [0, 200]
)

fig_step.show()

In [ ]:
# Visualize & Check perturbation value.
def eval_perturb(x, perturb_ini, param_gamma):
    perturb = perturb_ini / (x**param_gamma)
    return perturb


iter_opti = list(range(1,201))
perturb = list(map(lambda x: eval_perturb(x, perturb_ini= 2, param_gamma=0.1), iter_opti))

fig_perturb = go.Figure()

fig_perturb.add_trace(
    go.Scatter(
        x= iter_opti,
        y= perturb,
        line_color = "#000000",
    )
)

fig_perturb.update_layout(
    title= "Perturbing Changes",
    xaxis_title= "Number Iteration",
    yaxis_title= "Perturbing Value [NrVeh/hr]",
    width= 500,
    height= 350,
    template= tempelete_01_white,
)

fig_perturb.update_xaxes(
    range= [0, 200]
)

fig_perturb.show()

In [ ]:
# Create SPSA object.
fdsa_01 = fdsa(
    in_fl_step_ini= 100, in_fl_param_a= 9, in_fl_param_alpha= 0.3,
    in_fl_perturb_ini= 2, in_fl_param_gamma= 0.1, 
    in_int_iter_gradi= 1, in_int_iter_opti= 100, in_int_seg_size= 5,
    in_lv_seg_step= False, in_lv_seg_perturb= False, in_lv_min_bound= True,
)

In [ ]:
# Get iterating numbers.
int_nr_iter_opti = fdsa_01.int_iter_opti
int_nr_iter_gradi = fdsa_01.int_iter_gradi

In [ ]:
# Import true od matrix.
str_path_od_true = "./data_tabular/true_od_v02.csv"
fdsa_01.read_od_true(str_path_od_true)

In [ ]:
# Import biased od matrix.
str_path_od_base = "./data_tabular/biased_od_v02.csv"
fdsa_01.read_od_base(str_path_od_base)

In [ ]:
# Import true flow.
str_path_flow_true = "./data_tabular/edge_flow_true_v02.csv"
fdsa_01.read_true_flow(str_path_flow_true)

In [ ]:
# Load GNN model and trained weighting factor.
str_path_gnn = "gnn_model/GATv2_3CONV_v03_weight.pth"
str_path_adj_idx = "data_tabular/node_adg_matrix/arr_adj_idx.npy"
fdsa_01.load_gnn_model(str_path_gnn, str_path_adj_idx)

In [ ]:
# MAIN CALCULATION: FDSA (Finite Difference Stochastic Approximation)
# Perturbation is done by Graph Neural Network model.

# Measure time.
start_time = time.time()
# Boost gradient?
lv_bst_gradi = False
fac_bst_gradi = 5
# Which method for simulation?
lv_sim_gnn_min = True

# LOOP_1, Optimizing Iteration.
for iter_opti in range(1, int_nr_iter_opti + 1):
    
    # Parameters update for optimisation loop.
    # Step size (a_k), Perturbation coefficient (c_k).
    fdsa_01.param_update(iter_opti)
    
    # Create empty gradient matrix.
    arr_gradi_tmp1 = np.zeros(fdsa_01.tup_dim_od)
    
    # LOOP_2, Gradient Estimation. Iterating over perturbation generator.
    for lst_perturb in fdsa_01.gen_pertub_dfs():
        # Retrieve index and column number of element.
        idx_tmp2 = lst_perturb[2]
        col_tmp2 = lst_perturb[3]
        # Retrieve perturbed matrices.
        df_od_perturb_plus_tmp2 = lst_perturb[0]
        df_od_perturb_minus_tmp2 = lst_perturb[1]
        # Simulation with perturbed matrices.
        df_flow_perturb_plus_tmp2 = fdsa_01.sim_gnn_get_flow(iter_opti, 0, df_od_perturb_plus_tmp2)
        df_flow_perturb_minus_tmp2 = fdsa_01.sim_gnn_get_flow(iter_opti, 0, df_od_perturb_minus_tmp2)
        # Calculate nRMSE with true flow matrices.
        gof_plus = fdsa_01.n_Rmse_iq(fdsa_01.df_true_flow, df_flow_perturb_plus_tmp2)
        gof_minus = fdsa_01.n_Rmse_iq(fdsa_01.df_true_flow, df_flow_perturb_minus_tmp2)
        # Calculate gradient for specific element.
        if fdsa_01.lv_seg_perturb:
            fl_perturb_tmp2 = fdsa_01.df_perturb_seg.iat[idx_tmp2, col_tmp2]
            fl_gradi_tmp2 = (gof_plus - gof_minus) / (2*fl_perturb_tmp2)
        else:            
            fl_gradi_tmp2 = (gof_plus - gof_minus) / (2*fdsa_01.fl_perturb)
        # Boost gradient if required.
        if lv_bst_gradi:
            fl_gradi_tmp2 = fl_gradi_tmp2 * fac_bst_gradi
        # Update gradient matrix.
        arr_gradi_tmp1[idx_tmp2, col_tmp2] = fl_gradi_tmp2
        
    # Store gradient matrix in internal list.
    fdsa_01.lst_gradi.append(arr_gradi_tmp1)
    print(f"    Gradient esimation is ready. Iteration_{iter_opti}.")
    
    # Minimizing with step value.
    fdsa_01.minimization_loss(iter_opti)
    if lv_sim_gnn_min:
        df_flow_min = fdsa_01.sim_gnn_get_flow(iter_opti, 999, fdsa_01.df_od)
    else:
        df_flow_min = fdsa.sim_sumo_get_flow(iter_opti, 999, fdsa_01.df_od)    
    nRmse_min = fdsa.n_Rmse(fdsa_01.df_true_flow, df_flow_min)
    fdsa_01.lst_nRmse.append(nRmse_min)
    
    # Make the best record.
    if iter_opti == 1:
        fdsa_01.nRmse_best = nRmse_min
        fdsa_01.df_flow_best = df_flow_min
        fdsa_01.df_od_best = fdsa_01.df_od
    else:
        if nRmse_min < fdsa_01.nRmse_best:
            fdsa_01.nRmse_best = nRmse_min
            fdsa_01.df_flow_best = df_flow_min
            fdsa_01.df_od_best = fdsa_01.df_od
    
    # Make time stamp.
    time_epoch = time.time() - start_time
    fdsa_01.lst_time_epoch.append(int(time_epoch))     
    
    print(f"    Iteration_{iter_opti}/{int_nr_iter_opti}: nRMSE {nRmse_min}")
    print(f"    Best nRMSE so far: {fdsa_01.nRmse_best}")

In [ ]:
# Store FDSA result dataframe.
dic_hist_fdsa = {
    "Iteration"     : range(1, int_nr_iter_opti + 1),
    "Time"          : fdsa_01.lst_time_epoch,
    "Perturbation"  : fdsa_01.lst_perturb,
    "Step"          : fdsa_01.lst_step,
    "NRMSE"         : fdsa_01.lst_nRmse,
}
df_hist_fdsa = pd.DataFrame(dic_hist_fdsa)
df_hist_fdsa.to_csv("./reporting/result_fdsa.csv")

In [ ]:
# Store best OD matrix and corresponding traffic flow result.
fdsa_01.df_od_best.to_csv("./reporting/result_fdsa_od_best.csv")
fdsa_01.df_flow_best.to_csv("./reporting/result_fdsa_yflow_best.csv")

In [ ]:
# Check how much optimized OD matrix close to true one.
# Metric is MAPE.
from torchmetrics import MeanAbsolutePercentageError as MAPE
import torch

ts_od_best = torch.from_numpy(fdsa_01.df_od_best.astype(float).values)
ts_od_true = torch.from_numpy(fdsa_01.df_od_true.astype(float).values)

clc_mape = MAPE()
clc_mape(ts_od_best, ts_od_true)

In [ ]:
# When the best result was calculated?
df_hist_fdsa[
    df_hist_fdsa.NRMSE == df_hist_fdsa.NRMSE.min()
]

In [ ]:
# NRMES (Normalized RMES) vs Iteration plot.
fig_iter = go.Figure()

fig_iter.add_trace(
    go.Scatter(
        x= df_hist_fdsa["Iteration"],
        y= df_hist_fdsa["NRMSE"],
        line_color = "#000000",
    )
)

fig_iter.update_layout(
    title= "FDSA with GNN",
    xaxis_title= "Number Iteration",
    yaxis_title= "NRMES with y_mean",
    width= 500,
    height= 350,
    template= tempelete_01_white,
)

fig_iter.update_xaxes(
    range= [0, df_hist_fdsa["Iteration"].max() + 5]
)

fig_iter.update_yaxes(
    range= [0,df_hist_fdsa["NRMSE"].max()]
)

fig_iter.show()

In [ ]:
# True Flow vs Best Flow plot.

arr_flow_true_flatten = fdsa_01.df_true_flow.values.flatten()
arr_flow_fdsa_flatten = fdsa_01.df_flow_best.values.flatten()

fig_counts = go.Figure()

fig_counts.add_trace(
    go.Scatter(
        x= arr_flow_true_flatten,
        y= arr_flow_fdsa_flatten,
        mode= "markers",
        marker_symbol= "circle-open",
        marker_color= "#000000",   
    )
)

fig_counts.add_traces(
    go.Scatter(
        x= [0,500],
        y= [0,500],
        mode= "lines",        
        marker_color= "#fc4040",        
    )
)

fig_counts.update_layout(
    title= "FDSA with GNN",
    xaxis_title= "True Counts",
    yaxis_title= "FDSA Counts",
    width= 500,
    height= 500,
    showlegend= False,
    template= tempelete_01_white,
)

fig_counts.update_xaxes(
    range= [0, 500]
)

fig_counts.update_yaxes(
    range= [0,500]
)

fig_counts.show()